# GCP Demos

### Capstone Project GCP Location: https://console.cloud.google.com/home/dashboard?project=project2-2019-259821

### Geo info: http://donnees.ville.montreal.qc.ca/dataset/geobase
https://robertorocha.info/how-i-made-the-montreal-street-history-map/

In [1]:
import networkx as nx
import pandas as pd
import requests
import json
from math import sin, cos, sqrt, atan2, radians
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
with open(r"./geojson_dump.json", 'r') as f:    
    data = f.read()

In [3]:
data_json = json.loads(data)

In [4]:
def distance_in_km(e1, e2):

    # approximate radius of earth in km
    R = 6373.0
    lat1 = radians(e1[1])
    lon1 = radians(e1[0])
    lat2 = radians(e2[1])
    lon2 = radians(e2[0])
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    
    return distance

In [5]:
d = []
for features in data_json['features']:
    d.append(features['properties']['ARR_DRT'])
print(set(d))

{'Lachine', 'Outremont', 'Ville-Marie', 'Saint-Léonard', 'Anjou', 'Le Sud-Ouest', 'Rosemont-La Petite-Patrie', 'Verdun', 'Pierrefonds-Roxboro', 'Côte-des-Neiges-Notre-Dame-de-Grâce', 'N/A', 'Villeray-Saint-Michel-Parc-Extension', 'LaSalle', 'Le Plateau-Mont-Royal', 'Ahuntsic-Cartierville', 'Mercier-Hochelaga-Maisonneuve', 'Saint-Laurent', 'Rivière-des-Prairies-Pointe-aux-Trembles', 'Montréal-Nord', "L'Île-Bizard-Sainte-Geneviève"}


In [6]:
G = nx.Graph()
for feature in data_json['features']:
    # if feature['properties']['ARR_DRT'] == "Le Sud-Ouest":
    for idx, coord in enumerate(feature['geometry']['coordinates']):
        coord = tuple(coord)
        if coord not in G:
            G.add_node(coord, **feature['properties'])
        if idx != 0:
            G.add_edge(last_node, coord, length=distance_in_km(coord, last_node))
        last_node = coord

In [7]:
def longest_node_distance():
    max_dist = 0
    for idx, start in enumerate(list(G.nodes(data=True))):
        for end in list(G.nodes(data=True)):
            if start != end:
                try:
                    dist = nx.shortest_path_length(G,source=start[0], target=end[0], weight='length')
                except nx.NetworkXNoPath:
                    continue
                if dist > max_dist:
                    max_dist = dist
                    start_node = start
                    end_node = end
        break
    return start_node, end_node, max_dist

In [8]:
# pos = nx.spring_layout(G)
# nx.draw_networkx_nodes(G, pos, cmap=plt.get_cmap('jet'), node_size = 0)
# #nx.draw_networkx_labels(G, pos)
# nx.draw_networkx_edges(G, pos, edgelist=G.edges())

In [9]:
fire_stations = pd.read_csv(r"C:\Users\olive\Documents\GitHub\MontrealFireSafetyProject\casernes.csv")
fire_stations.head()

,CASERNE,NO_CIVIQUE,RUE,LATITUDE,LONGITUDE,ARRONDISSEMENT,VILLE,DATE_DEBUT,DATE_FIN,MTM8_X,MTM8_Y
0,3,256,rue Young,45.493454,-73.560172,LE SUD-OUEST,NaN,2015-01-01T07:00:00,NaN,300097.0,5039283.0
1,15,1255,rue de la Sucrerie,45.484419,-73.560917,LE SUD-OUEST,NaN,2015-01-01T07:00:00,NaN,300038.0,5038279.0
2,23,523,place Saint-Henri,45.477925,-73.585251,LE SUD-OUEST,NaN,2015-01-01T07:00:00,NaN,298135.0,5037559.0
3,33,6040,boulevard Monk,45.457841,-73.595450,LE SUD-OUEST,NaN,2015-01-01T07:00:00,NaN,297335.0,5035328.0
4,9,8100,boulevard Saint-Michel,45.563748,-73.610169,VILLERAY-SAINT-MICHEL-PARC-EXTENSION,NaN,2019-05-06T07:00:00,NaN,296200.0,5047099.0


In [10]:
fs_update = dict(zip(G.nodes, [{'FIRE_STATION': False} for x in G.nodes]))
nx.set_node_attributes(G, fs_update)
for index, row in fire_stations.iterrows():
    min_dist = 1000
    f_long, f_lat = row['LONGITUDE'], row['LATITUDE']
    for node in list(G.nodes):
        dist = distance_in_km(node, (f_long, f_lat))
        if dist < min_dist:
            min_dist = dist
            closest_node = node
    nx.set_node_attributes(G, {closest_node: {'FIRE_STATION': True}})

In [11]:
# fs_nodes = [x for x, y in G.nodes(data=True) if y['FIRE_STATION']==True]
# idx = 0
# for node, _ in G.nodes(data=True):
#     idx += 1
#     print(idx, '/', len(G.nodes))
#     min_dist = 1000
#     for fs in fs_nodes:
#         try:
#             dist = nx.shortest_path_length(G, source=fs, target=node, weight='length')
#         except nx.NetworkXNoPath:
#             continue
#         if dist < min_dist:
#             min_dist = dist
#     nx.set_node_attributes(G, {node: {'DIST_TO_FS': min_dist}})

In [12]:
data = []
for node in G.nodes(data=True):
    data.append({'Longitude': node[0][0], 'Latitude': node[0][1], **node[1]})
res = pd.DataFrame(data)
res.to_csv(r"node_data.csv")